# importando tabula e arquivos pdf
- importante que os PDFs estejam todos na mesma página dos arquivos

In [1]:
import tabula
import pandas as pd

lista_suap = tabula.read_pdf("suap/2023_08_RMA_suap.pdf", pages="1")
mes = '2023_08_RMA'
# lista em dataframe
df_suap = lista_suap[0]

# removendo linhas com valores nulos
df_suap = df_suap.dropna()
display(df_suap)

df_suap = df_suap[:-1]

# removendo e renomeando colunas
df_suap = df_suap.drop(columns=['Nome', 'Estoque', 'Entrada'])
df_suap['ED'] = df_suap['Cod.']
df_suap['saida'] = df_suap['Saída']
df_suap['saldo_suap'] = df_suap['Estoque.1']
df_suap = df_suap.drop(columns=['Cod.', 'Estoque.1', 'Saída'])

# substituindo caracteres para a conversão em numero
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(".", "", regex=True)
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(",", ".", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(".", "", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(",", ".", regex=True)


df_suap['saldo_suap'] = pd.to_numeric(df_suap['saldo_suap'])
df_suap['saida'] = pd.to_numeric(df_suap['saida'])
print("{:.2f}".format(df_suap['saldo_suap'].sum()))
print(df_suap.info())
display(df_suap)

Got stderr: Sep 06, 2023 3:23:49 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Sep 06, 2023 3:23:49 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



,Cod.,Nome,Estoque,Entrada,Saída,Estoque.1
1,07,GENEROS DE ALIMENTACAO,"0,00","748,00","748,00","0,00"
2,11,MATERIAL QUIMICO,"0,00","3.945,00","3.200,00","745,00"
3,14,MATERIAL EDUCATIVO E ESPORTIVO,"31.434,64","0,00","1.686,59","29.748,05"
4,16,MATERIAL DE EXPEDIENTE,"54.204,35","0,00","5,76","54.198,59"
5,17,MATERIAL DE PROCESSAMENTO DE DADOS,"13.477,99","0,00","0,00","13.477,99"
6,19,MATERIAL DE ACONDICIONAMENTO E,"2.223,00","0,00","0,00","2.223,00"
8,21,MATERIAL DE COPA E COZINHA,"6.582,29","0,00","0,00","6.582,29"
9,22,MATERIAL DE LIMPEZA E PROD. DE,"12.783,98","0,00","124,00","12.659,98"
11,23,"UNIFORMES, TECIDOS E AVIAMENTOS","22.041,79","0,00","0,00","22.041,79"
12,24,MATERIAL P/ MANUT.DE BENS,"31.889,17","0,00","0,00","31.889,17"


235226.56
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 23
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ED          18 non-null     object 
 1   saida       18 non-null     float64
 2   saldo_suap  18 non-null     float64
dtypes: float64(2), object(1)
memory usage: 576.0+ bytes
None


,ED,saida,saldo_suap
1,07,748.00,0.00
2,11,3200.00,745.00
3,14,1686.59,29748.05
4,16,5.76,54198.59
5,17,0.00,13477.99
6,19,0.00,2223.00
8,21,0.00,6582.29
9,22,124.00,12659.98
11,23,0.00,22041.79
12,24,0.00,31889.17


In [2]:
# parâmetros lattice=True e guess=False para conseguir reconhecer e montar tabelas em arquivos com formatação diferente

lista_siafi = tabula.read_pdf("siafi/2023_08_RMA_siafi.pdf", guess=False, pages="1-2")

tabela1 = lista_siafi[0]
tabela1.columns = tabela1.iloc[7]
tabela1 = tabela1[8:20]

tabela2 = lista_siafi[1]
tabela2.columns = tabela2.iloc[7]
tabela2 = tabela2[8:15]

# Renomeando coluna por causa do caracter especial $

tabela1.columns = ['conta']
tabela2.columns = ['conta']

# unindo tabelas pela coluna conta
df_siafi = pd.merge(tabela1, tabela2, how = 'outer', on = 'conta')



# fatiando coluna para obter colunas ED e Saldo
df_siafi['ED'] = df_siafi['conta'].str[:4]
df_siafi['saldo_siafi'] = df_siafi['conta'].str[4:-1]

# removendo coluna ajustada
df_siafi = df_siafi.drop(columns=['conta'])

# utilize aqui para dropar as linhas extras quando some ou adiciona um novo ED
display(df_siafi)
df_siafi = df_siafi.drop(index=[15,16,17,18])

# substituindo caracteres para a conversão em numero
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(".", "", regex=True)
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(",", ".", regex=True)
df_siafi['ED'] = df_siafi['ED'].str.replace("P ", "", regex=True)
df_siafi['saldo_siafi'] = pd.to_numeric(df_siafi['saldo_siafi'])

print("{:.2f}".format(df_siafi['saldo_siafi'].sum()))
display(df_siafi)
print(df_siafi.info())

,ED,saldo_siafi
0,P 14,"31.434,64"
1,P 16,"54.204,35"
2,P 17,"13.477,99"
3,P 19,"2.223,00"
4,P 21,"6.582,29"
5,P 22,"12.783,98"
6,P 23,"22.041,79"
7,P 24,"31.889,17"
8,P 25,"199,60"
9,P 26,"19.109,10"


236963.93


,ED,saldo_siafi
0,14,31434.64
1,16,54204.35
2,17,13477.99
3,19,2223.00
4,21,6582.29
5,22,12783.98
6,23,22041.79
7,24,31889.17
8,25,199.60
9,26,19109.10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ED           15 non-null     object 
 1   saldo_siafi  15 non-null     float64
dtypes: float64(1), object(1)
memory usage: 360.0+ bytes
None


In [3]:
# unindo data frames do suap e siafi em uma unica tabela
df_rma = pd.merge(df_suap, df_siafi, how = 'inner', on='ED')
df_rma = df_rma.reindex(columns=['ED','saldo_suap', 'saida', 'saldo_siafi'])

# calculando a diferença entre colunas 
df_rma['diferenca'] = df_rma['saldo_suap'] - df_rma['saldo_siafi']

# definindo a ação para cafa resultado
acao = []
for valor in df_rma['diferenca']:
    
    if valor > 0:
        baixa = "baixa no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "baixa no siafi"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

df_rma['acao'] = acao

display(df_rma)
df_rma.to_clipboard()
df_rma.to_excel(f'saida/{mes}.xlsx')

,ED,saldo_suap,saida,saldo_siafi,diferenca,acao
0,14,29748.05,1686.59,31434.64,-1686.59,baixa no siafi
1,16,54198.59,5.76,54204.35,-5.76,baixa no siafi
2,17,13477.99,0.00,13477.99,0.00,adequado
3,19,2223.00,0.00,2223.00,0.00,adequado
4,21,6582.29,0.00,6582.29,0.00,adequado
5,22,12659.98,124.00,12783.98,-124.00,baixa no siafi
6,23,22041.79,0.00,22041.79,0.00,adequado
7,24,31889.17,0.00,31889.17,0.00,adequado
8,25,199.60,0.00,199.60,0.00,adequado
9,26,19055.07,54.03,19109.10,-54.03,baixa no siafi
